In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/Feger/suger-pie-honey-pot

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/Feger/suger-pie-honey-pot


In [2]:
!pip install xlsxwriter==3.2.0

# Requirements

In [3]:
import os
import pandas as pd
import numpy as np

SAMPLE_PATH = "./annotations/samples.xlsx"
QUERIED_SAMPLE_PATH = "./annotations/queried_samples.xlsx"

In [4]:
# Load data from a CSV file into a DataFrame without setting any column as the index
df_data = pd.read_csv('./data/outputs/combined_w_session_info.csv', index_col=False)

# Group the data by 'session' and calculate the size of each group
# Then reset the index to turn it into a DataFrame and rename the size column to 'session-size'
grouped_sizes = df_data.groupby('session').size().reset_index(name='session-size')

# Merge the original DataFrame with the session sizes DataFrame on the 'session' column
# This adds the 'session-size' column to df_data. Use 'left' join to keep all rows from df_data
df_data = df_data.merge(grouped_sizes, on='session', how='left')

# Fill missing values in 'session-size' with 1, assuming sessions not grouped have size 1
df_data['session-size'] = df_data['session-size'].fillna(1)

# Fill missing values in 'session-size' with 1, assuming sessions not grouped have size 1
df_data['true-response'] = df_data['true-response'].fillna("")

# Fill missing 'session' values with -1, assigning a default identifier to missing sessions
df_data['session'] = df_data['session'].fillna(-1)

# Similarly, fill missing 'session-id' values with -1, providing a default for missing data
df_data['session-id'] = df_data['session-id'].fillna(-1)

# Sampling

In [5]:
def sample_n_per_group(df, n, random_seed=123456789):
    # Calculate the number of samples per 'base-command' group, rounding up
    normalized_counts = n * df["base-command"].value_counts(normalize=True)
    normalized_counts = np.ceil(normalized_counts) # Have at least one command sampled

    # Sample from each group based on the calculated counts
    sampled_df = df.groupby('base-command').apply(
        lambda cmd: cmd.sample(
            n=int(min(len(cmd), normalized_counts.loc[cmd.name])),
            random_state=random_seed
        )
    ).reset_index(drop=True)
    if not -1 in sampled_df["session"].unique():
        sampled_df = df[df["session"].isin(sampled_df["session"].unique())]
    return sampled_df

In [6]:
def aggregate_data(df_sample, df_data):
    # Step 1: Concatenate df_sample and df_data, and prepare the data
    df_sample["source"] = "Sample"
    df_data["source"] = "Original"
    df = pd.concat([df_sample, df_data])

    df["length"] = df['commands'].apply(lambda x: len(str(x)))

    # Aggregating various statistics together for each dataset-name and source
    aggregations = {
        "request-id": "nunique",
        "base-command": "nunique",
        "length": "mean",
        "true-response": lambda x: (x == "").sum()
    }
    df_aggregated = df.groupby(["dataset-name", "source"]).agg(aggregations).reset_index()
    df_aggregated.columns = ["dataset-name", "source", "size", "base-commands", "mean_command_length", "empty_responses"]

    # Function to calculate session stats, including sessions below size 50
    def calculate_session_stats(group):
        #print((group["session-size"]<=50).sum()/group.shape[0])
        session_sizes = group.groupby("session").size()
        below_50 = round(100*(group["session-size"]<=50).sum()/group.shape[0], 2)
        mean_size = session_sizes.mean()
        min_size = session_sizes.min()
        max_size = session_sizes.max()
        return pd.Series([mean_size, min_size, max_size, below_50], index=["mean_session_size", "min_session_size", "max_session_size", "sessions_below_50"])

    # Apply the function to calculate session stats
    session_stats = df[df["session"] != -1].groupby(["dataset-name", "source"]).apply(calculate_session_stats).reset_index()

    # Merge aggregated data with session size data
    df_total = df_aggregated.merge(session_stats, on=["dataset-name", "source"], how="left")

    # Rounding and filling NAs
    df_total["mean_command_length"] = df_total["mean_command_length"].round(2)
    df_total["mean_session_size"] = df_total["mean_session_size"].fillna(1).round()

    # Overall statistics for each source without considering dataset-name
    df_overall_aggregated = df.groupby("source").agg(aggregations).reset_index()
    df_overall_aggregated.columns = ["source", "size", "base-commands", "mean_command_length", "empty_responses"]

    # Calculate overall session stats
    overall_session_stats = df[df["session"] != -1].groupby("source").apply(calculate_session_stats).reset_index()

    # Merge overall aggregated data with session size data
    df_overall_total = df_overall_aggregated.merge(overall_session_stats, on="source", how="left")

    # Rounding and filling NAs for overall data
    df_overall_total["mean_command_length"] = df_overall_total["mean_command_length"].round(2)
    df_overall_total["mean_session_size"] = df_overall_total["mean_session_size"].fillna(1).round()
    df_overall_total["dataset-name"] = "total"

    # Concatenate dataset-specific and overall analyses
    final_df = pd.concat([df_total, df_overall_total])
    final_df["min_session_size"] = final_df["min_session_size"].fillna(1).round()
    final_df["max_session_size"] = final_df["max_session_size"].fillna(1).round()
    final_df["sessions_below_50"] = final_df["sessions_below_50"].fillna(0).astype(float)

    return final_df

In [7]:
df_sample = df_data[df_data["session-size"] <= 50].groupby('dataset-name').apply(lambda x: sample_n_per_group(x, 200))
df_sample = df_sample.sort_values(by=['session', 'session-id'], ascending=[True, True])
# Assuming df_sample and df_data are already defined and prepared
df_final = aggregate_data(df_sample, df_data)
df_final

,dataset-name,source,size,base-commands,mean_command_length,empty_responses,mean_session_size,min_session_size,max_session_size,sessions_below_50
0,halle,Original,2483,85,468.09,872,5.0,1.0,644.0,62.95
1,halle,Sample,943,59,317.45,305,7.0,1.0,32.0,100.00
2,nl2bash,Original,6056,183,48.95,0,1.0,1.0,1.0,0.00
3,nl2bash,Sample,334,183,47.25,0,1.0,1.0,1.0,0.00
4,prague,Original,129,49,13.43,0,1.0,1.0,1.0,0.00
5,prague,Sample,129,49,13.43,0,1.0,1.0,1.0,0.00
0,total,Original,8668,251,168.49,872,5.0,1.0,644.0,62.95
1,total,Sample,1406,230,225.37,305,7.0,1.0,32.0,100.00


In [8]:
if not os.path.exists(SAMPLE_PATH):
    df_sample.to_excel(SAMPLE_PATH, engine='xlsxwriter', index=False)

# Task Generation

In [9]:
def merge_llm_responses(df, df_llm):
    assert sorted(df["dataset-name"].unique()) == sorted(df_llm["dataset-name"].unique())
    # assert that the llm has all samples covered.
    for dataset in df["dataset-name"].unique():
        # implicitly checks if the sessions are sorted in the same way
        assert (df_llm[df_llm["dataset-name"] == dataset]["request-id"].values == df[df["dataset-name"] == dataset]["request-id"].values).all()

    df = df.merge(df_llm[['request-id', 'generated-response']], on='request-id', how='left')
    df['generated-response'] = df['generated-response'].fillna("")
    df['true-response'] = df['true-response'].fillna("")
    return df

def suffle_dataframe(df, random_seed=42):
    # Group by 'session' while preserving the order within each group
    groups = [df_ for _, df_ in df.groupby('session')]

    # Shuffle the groups without changing the order within each group
    np.random.seed(random_seed)  # For reproducibility
    shuffled_indices = np.random.permutation(len(groups))

    # Use the shuffled indices to reorder the groups
    shuffled_groups = [groups[i] for i in shuffled_indices]
    # Concatenate the shuffled groups back into a DataFrame
    shuffled_df = pd.concat(shuffled_groups).reset_index(drop=True)

    assert (shuffled_df.sort_values(by='request-id').values == df.sort_values(by='request-id').values).all()

    return shuffled_df

def write_ann_files(df, path, file_name, random_seed=42):
    df_halle = suffle_dataframe(df[df["dataset-name"]=="halle"], random_seed=random_seed)
    df_nl2bash = suffle_dataframe(df[df["dataset-name"]=="nl2bash"], random_seed=random_seed)
    df_prague = suffle_dataframe(df[df["dataset-name"]=="prague"], random_seed=random_seed)

    columns = ["request-id", "session", "session-id", "commands", "true-response", "generated-response", "believable"]

    if not os.path.exists(path+f"/halle.xlsx"):
        df_halle[columns].to_excel(path+f"/halle.xlsx", engine='xlsxwriter', index=False)
    if not os.path.exists(path+f"/nl2bash.xlsx"):
        df_nl2bash[columns].to_excel(path+f"/nl2bash.xlsx", engine='xlsxwriter', index=False)
    if not os.path.exists(path+f"/prague.xlsx"):
        df_prague[columns].to_excel(path+f"/prague.xlsx", engine='xlsxwriter', index=False)

In [10]:
df_merged = merge_llm_responses(pd.read_excel(SAMPLE_PATH), pd.read_excel(QUERIED_SAMPLE_PATH))
df_merged['believable'] = "ANSWER"

# For annotators
for file_name in range(1, 7):
    PATH = f"./annotations/annotator_{file_name}"
    if not os.path.exists(PATH):
        os.mkdir(PATH)
    write_ann_files(df_merged, PATH, file_name, file_name)

# For authors
for file_name in ["marc", "simon"]:
    PATH = f"./annotations/annotator_{file_name}"
    if not os.path.exists(PATH):
        os.mkdir(PATH)
    write_ann_files(df_merged, PATH, file_name, 42)